<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Vanishing/Exploding-Gradients" data-toc-modified-id="Vanishing/Exploding-Gradients-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Vanishing/Exploding Gradients</a></span><ul class="toc-item"><li><span><a href="#Xavier-initializer-(aka-Glorot-uniform-initializer)" data-toc-modified-id="Xavier-initializer-(aka-Glorot-uniform-initializer)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Xavier initializer (aka Glorot uniform initializer)</a></span></li><li><span><a href="#He-initializer" data-toc-modified-id="He-initializer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>He initializer</a></span></li><li><span><a href="#Nonsaturating-Activation-Functions" data-toc-modified-id="Nonsaturating-Activation-Functions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Nonsaturating Activation Functions</a></span><ul class="toc-item"><li><span><a href="#Exponential-linear-unit-(ELU)" data-toc-modified-id="Exponential-linear-unit-(ELU)-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Exponential linear unit (ELU)</a></span></li><li><span><a href="#Leaky-ReLu" data-toc-modified-id="Leaky-ReLu-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Leaky ReLu</a></span></li></ul></li><li><span><a href="#Batch-normalization" data-toc-modified-id="Batch-normalization-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Batch normalization</a></span></li><li><span><a href="#Gradient-Clipping" data-toc-modified-id="Gradient-Clipping-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Gradient Clipping</a></span></li></ul></li><li><span><a href="#Transfer-Learning" data-toc-modified-id="Transfer-Learning-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Transfer Learning</a></span><ul class="toc-item"><li><span><a href="#Build-sample-model-to-learn-from" data-toc-modified-id="Build-sample-model-to-learn-from-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Build sample model to learn from</a></span></li><li><span><a href="#Transfer-learning---Method-one" data-toc-modified-id="Transfer-learning---Method-one-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Transfer learning - Method one</a></span></li><li><span><a href="#Transfer-learning---Method-two" data-toc-modified-id="Transfer-learning---Method-two-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Transfer learning - Method two</a></span></li><li><span><a href="#Freezing-the-lower-layers" data-toc-modified-id="Freezing-the-lower-layers-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Freezing the lower layers</a></span></li><li><span><a href="#Caching-frozen-layers" data-toc-modified-id="Caching-frozen-layers-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Caching frozen layers</a></span></li><li><span><a href="#Altering-layers" data-toc-modified-id="Altering-layers-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Altering layers</a></span></li></ul></li></ul></div>

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# Setup vars for the MINST data set
inputs = 28 * 28    # image dim in pixels
hidden1 = 300
hidden2 = 100
hidden3 = 50
hidden4 = 50
hidden5 = 50
outputs = 10

lr = 0.01
epochs = 20
batchSize = 50

mninst = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# Vanishing/Exploding Gradients

## Xavier initializer (aka Glorot uniform initializer)

This is the default init for the `tf.layers.dense` method.

## He initializer 

In [5]:
# Reset the TF CG
tf.reset_default_graph()

In [6]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    # He initializer
    heInit = tf.contrib.layers.variance_scaling_initializer()
    
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu, kernel_initializer = heInit)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [7]:
# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([opt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.92 Test Acc:  0.9055
10 Train Acc:  0.98 Test Acc:  0.9605
 
FINAL ::  Train Acc:  0.98 Test Acc:  0.9699


## Nonsaturating Activation Functions

Helps to mitigate the `dying ReLu` problem.

### Exponential linear unit (ELU)

In [8]:
# Clear everything out
x = y = layer1 = layer2 = yH = entropy = loss = opt = correct = None
accuracy = init = saver = accTrain = accTest = savePath = sess = None
entropy = None

# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [9]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    # Use ELU
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.elu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.elu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [10]:
# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([opt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.9 Test Acc:  0.9023
10 Train Acc:  0.96 Test Acc:  0.949
 
FINAL ::  Train Acc:  0.98 Test Acc:  0.9625


### Leaky ReLu

In [11]:
# Clear everything out
x = y = layer1 = layer2 = yH = entropy = loss = opt = correct = None
accuracy = init = saver = accTrain = accTest = savePath = sess = None
entropy = None

# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [12]:
# Reset the TF CG
tf.reset_default_graph()

In [13]:
# Write the leaky ReLu function
def leakyReLu(z, name = None):
    return tf.maximum(0.01 * z, z, name = name)

In [14]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    # Use leaky ReLu
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = leakyReLu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = leakyReLu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [15]:
# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([opt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  0.94 Test Acc:  0.9041
10 Train Acc:  1.0 Test Acc:  0.9595
 
FINAL ::  Train Acc:  0.98 Test Acc:  0.9714


## Batch normalization

In [16]:
# Clear everything out
x = y = layer1 = layer2 = yH = entropy = loss = opt = correct = None
accuracy = init = saver = accTrain = accTest = savePath = sess = None
entropy = None

# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [17]:
# Write a partial, so we can have cleaner code  :)
from functools import partial

# We'll use this to control batch normalization during training... 
# Set it to TRUE when training, and FALSE otherwise
training = tf.placeholder_with_default(False, shape = (), name = 'training')

batchNormLayer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)

In [18]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne")
    batchNorm1 = batchNormLayer(layer1)
    batchNorm1Activation = tf.nn.elu(batchNorm1)
    
    layer2 = tf.layers.dense(batchNorm1Activation, hidden2, name = "hLayerTwo")
    batchNorm2 = batchNormLayer(layer2)
    batchNorm2Activation = tf.nn.elu(batchNorm2)
    
    
    yHBeforeBatchNorm = tf.layers.dense(batchNorm2Activation, outputs, name = "yH")
    yH = batchNormLayer(yHBeforeBatchNorm)   
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [19]:
# Execute the TF CG

# New operations added from the batch normalization
updateOps = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            # Note that training = TRUE
            sess.run([opt, updateOps], feed_dict = {training: True, x: xBatch, y: yBatch})
        # Note that training = FALSE by default
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)

0 Train Acc:  1.0 Test Acc:  0.925
10 Train Acc:  1.0 Test Acc:  0.9744
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.9779


## Gradient Clipping

In [20]:
# Clear everything out
x = y = layer1 = layer2 = yH = entropy = loss = opt = correct = None
accuracy = init = saver = accTrain = accTest = savePath = sess = None
entropy = batchNorm1 = batchNorm1Activation = batchNorm2 = batchNorm2Activation = None
yHBeforeBatchNorm = None

# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [21]:
# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    yH = tf.layers.dense(layer2, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    # Gradient clipping threshold
    threshold = 1.0
    
    # Setup optimizer to use gradient clipping
    opt =  tf.train.GradientDescentOptimizer(lr)
    gradVars = opt.compute_gradients(loss)
    cappedGradVars = [(tf.clip_by_value(grad, -threshold, threshold), var)
                     for grad, var in gradVars]
    clippedOpt = opt.apply_gradients(cappedGradVars)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

# Save the trained model parameters to disk
saver = tf.train.Saver()

In [22]:
# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([clippedOpt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)
    
    # Save the trained model
    savePath = saver.save(sess, "./DNN-TensorFlow-Gradient-Clipping.ckpt")

0 Train Acc:  0.88 Test Acc:  0.9012
10 Train Acc:  0.96 Test Acc:  0.9592
 
FINAL ::  Train Acc:  0.98 Test Acc:  0.9708


# Transfer Learning

## Build sample model to learn from

In [23]:
# Reset the TF CG
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [24]:
# Create and save base model we want to use later on for transfer learning

# Build TF CG
x = tf.placeholder(tf.float32, shape = [None, inputs], name = 'x')
y = tf.placeholder(tf.int64, shape = [None], name = 'y')

# This will let TF take care of creating and init'ing the weights and creating the bias
with tf.name_scope("neuralNetEasy"):
    layer1 = tf.layers.dense(x, hidden1, name = "hLayerOne", activation = tf.nn.relu)
    layer2 = tf.layers.dense(layer1, hidden2, name = "hLayerTwo", activation = tf.nn.relu)
    layer3 = tf.layers.dense(layer2, hidden3, name = "hLayerThree", activation = tf.nn.relu)
    layer4 = tf.layers.dense(layer3, hidden4, name = "hLayerFour", activation = tf.nn.relu)
    layer5 = tf.layers.dense(layer4, hidden5, name = "hLayerFive", activation = tf.nn.relu)
    yH = tf.layers.dense(layer5, outputs, name = "yH")
    
with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = yH)
    loss = tf.reduce_mean(entropy, name = "loss")
    
with tf.name_scope("optimizer"):
    opt =  tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
# Eval the model's accuracy
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(yH, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name = 'accuracy')

init = tf.global_variables_initializer()

# Save our trained model parameters to disk
saver = tf.train.Saver()

# Method One:
#   Collect all the model operations in one spot to support model transfer learning
for op in (x, y, opt, accuracy):
    tf.add_to_collection("modelOps", op)

# Execute the TF CG
with tf.Session() as sess:
    init.run()
    
    for e in range(epochs):
        for i in range(mninst.train.num_examples // batchSize):
            xBatch, yBatch = mninst.train.next_batch(batchSize)
            sess.run([opt], feed_dict = {x: xBatch, y: yBatch})
        accTrain = accuracy.eval(feed_dict = {x: xBatch, y: yBatch})
        accTest = accuracy.eval(feed_dict = {x: mninst.test.images, y: mninst.test.labels})
        if e % 10 == 0:
            print(e, "Train Acc: ", accTrain, "Test Acc: ", accTest)
        
    print(" ")
    print("FINAL :: ", "Train Acc: ", accTrain, "Test Acc: ", accTest)
    
    # Save the CG, so we can restore and use it later on in another model
    savePath = saver.save(sess, "./DNN-Base-Pretrained-Model")

0 Train Acc:  0.94 Test Acc:  0.8969
10 Train Acc:  1.0 Test Acc:  0.9715
 
FINAL ::  Train Acc:  1.0 Test Acc:  0.9738


In [25]:
sess.close()

## Transfer learning - Method one

Attach to the TF CG components by utilizing the collection created by the orig. model author

In [26]:
# Clear everything out
x = y = layer1 = layer2 = yH = entropy = loss = opt = correct = None
accuracy = init = saver = accTrain = accTest = savePath = sess = None

# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [27]:
# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [28]:
sess2 = tf.Session() 

#First let's load meta graph and restore weights
saver2 = tf.train.import_meta_graph('./DNN-Base-Pretrained-Model.meta')
saver2.restore(sess2,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./DNN-Base-Pretrained-Model


In [29]:
# Attach to the TF CG components by utilizing the collection created by the orig. model author
# and pull out the first four
tf.get_collection("modelOps")
x2, y2, opt2, accuracy2 = tf.get_collection("modelOps")[0:4]

In [30]:
accTest2 = accuracy2.eval(session = sess2, feed_dict = {x2: mninst.test.images, y2: mninst.test.labels})
print(accTest2)


0.9738


In [31]:
# List the items in the collection
tf.get_collection("modelOps")

[<tf.Tensor 'x:0' shape=(?, 784) dtype=float32>,
 <tf.Tensor 'y:0' shape=(?,) dtype=int64>,
 <tf.Operation 'optimizer/GradientDescent' type=NoOp>,
 <tf.Tensor 'eval/accuracy:0' shape=() dtype=float32>]

## Transfer learning - Method two

Attach to the TF CG components by calling them specificly from the graph

In [32]:
# Clear everything out
x = y = layer1 = layer2 = yH = entropy = loss = opt = correct = None
accuracy = init = saver = accTrain = accTest = savePath = sess = None

# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [33]:
# Clear away all our previous work
tf.reset_default_graph()

# Confirm everything is cleared out
for op in tf.get_default_graph().get_operations():
    print(op.name)

In [34]:
# Create a new session
sess=tf.Session()    

#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('./DNN-Base-Pretrained-Model.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./DNN-Base-Pretrained-Model


In [35]:
# Attach to the TF CG components by calling them specificly from the graph
graph = tf.get_default_graph()
x = graph.get_tensor_by_name("x:0")
y = graph.get_tensor_by_name("y:0")

feed_dict = {x: mninst.test.images, y: mninst.test.labels}
accuracy = graph.get_tensor_by_name("eval/accuracy:0")

accTest = accuracy.eval(session = sess, feed_dict = {x: mninst.test.images, y: mninst.test.labels})
print(accTest)



0.9738


## Freezing the lower layers

## Caching frozen layers

## Altering layers